# 0.0. IMPORTS

In [1]:
import pickle

import numpy as np
import pandas as pd
import psycopg2

## 0.1. Helper Functions

In [2]:
def get_tables(table):
    '''This functions receives a list with table names to be loaded from the database'''
    # to create a df to store all loaded dfs
    df_tab = pd.DataFrame()
    
    # iterate over the list of tables, load each one, take the column names and create a final df
    for tab in table:
        sql = 'SELECT * FROM pa004.{}'.format(tab)
        cur.execute(sql)
        cols = [desc[0] for desc in cur.description] # saving the column names
        data = []
        for i in cur:
            data.append(i)
        df = pd.DataFrame(data, columns = cols)
        
        if 'id' in df_tab:
            df_tab = df_tab.merge(df, how = 'left', left_on = 'id', right_on = 'id')
        else:
            df_tab = df.copy()
    return df_tab

## 0.2. Collecting the Data

In [3]:
# creating parameters  to access the database
# excluded because it's private
params = dict(
    database='',
    user='',
    password='',
    host='',
    port=
)
# creating the connection
conn = psycopg2.connect(**params)

# creating the cursor
cur = conn.cursor()

**Let's find out how many tables there are**

In [4]:
sql = "select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)'"
cur.execute(sql)
tables = cur.fetchall()
table_names = []
for table, database in enumerate(tables, start=1):
    table_names.append(database[0])
    print(f'Table {table}: {database[0]}')

Table 1: users
Table 2: vehicle
Table 3: insurance


**Now, let's load them all and store them in a single dataframe.**

In [6]:
df_raw =  get_tables(table_names) # fucntion defined in section 0.1: Helper Functions
cur.close()
conn.close()

pickle.dump( df_raw, open( '../data/df_raw.pkl', 'wb') ) # saving the df as pickle format

In [9]:
df_raw = pickle.load(open('../data/df_raw.pkl', 'rb'))

In [11]:
df_raw.head()

,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,1,Male,44,28.0,26.0,1,> 2 Years,Yes,0,40454.0,217,1
1,2,Male,76,3.0,26.0,1,1-2 Year,No,0,33536.0,183,0
2,3,Male,47,28.0,26.0,1,> 2 Years,Yes,0,38294.0,27,1
3,4,Male,21,11.0,152.0,1,< 1 Year,No,1,28619.0,203,0
4,5,Female,29,41.0,152.0,1,< 1 Year,No,1,27496.0,39,0


# 1.0. STEP 1: DATA DESCRIPTION

_to do.._
